<a href="https://colab.research.google.com/github/viralbundela007/Datathon1/blob/main/Datathon1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer


import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow import feature_column

# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
train_df = pd.read_csv("/content/sample_data/train.csv")
#test_df = pd.read_csv("/content/sample_data/test.csv")
#solution_df=pd.read_csv("/content/sample_data/solution_template.csv")
train_df = train_df.reindex(np.random.permutation(train_df.index)) # shuffle the training set
y_train = train_df['metastatic_diagnosis_period']

In [5]:
cat_col = [col for col in train_df.columns if train_df[col].dtype == 'object']
print('Categorical columns :',cat_col)
# Numerical columns
num_col = [col for col in train_df.columns if train_df[col].dtype != 'object']
print('Numerical columns :',num_col)

Categorical columns : ['patient_race', 'payer_type', 'patient_state', 'Region', 'Division', 'patient_gender', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type']
Numerical columns : ['patient_id', 'patient_zip3', 'patient_age', 'bmi', 'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19', 'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s', 'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married', 'widowed', 'family_size', 'family_dual_income', 'income_household_median', 'income_household_under_5', 'income_household_5_to_10', 'income_household_10_to_15', 'income_household_15_to_20', 'income_household_20_to_25', 'income_household_25_to_35', 'income_household_35_to_50', 'income_household_50_to_75', 'income_household_75_to_100', 'income_household_100_to_150', 'income_household_150_over', 'income_household_six_figure', 'inc

In [6]:
train_df.drop(['patient_id','patient_gender', 'breast_cancer_diagnosis_desc','patient_state', 'Region', 'Division','patient_zip3','metastatic_first_novel_treatment_type','metastatic_diagnosis_period'], axis=1,inplace=True)

In [7]:
train_df.head()

,patient_race,payer_type,patient_age,bmi,breast_cancer_diagnosis_code,metastatic_cancer_diagnosis_code,metastatic_first_novel_treatment,population,density,age_median,...,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
12275,Other,MEDICARE ADVANTAGE,69,NaN,1749,C7951,NaN,20151.3,724.9,42.1,...,30.5,38.1,62.3,66.7,70.1,70.7,63.7,47.9,32.2,30.7
2416,Black,MEDICAID,38,NaN,C50911,C7951,NaN,58934.2,16936.6,36.2,...,37.2,46.5,64.2,69.4,76.4,76.8,68.9,54.8,41.1,36.5
11828,White,COMMERCIAL,58,NaN,C50012,C773,NaN,17003.1,917.0,40.3,...,29.8,40.4,60.4,63.8,69.5,69.9,62.6,49.2,34.1,30.4
8802,NaN,NaN,54,NaN,1749,C787,NaN,7149.1,159.8,45.3,...,31.7,39.8,60.3,64.6,72.4,71.9,63.9,48.9,35.1,29.9
9237,NaN,MEDICARE ADVANTAGE,42,NaN,C50112,C7800,NaN,16195.9,1070.4,38.6,...,37.2,46.0,69.8,73.1,74.2,74.3,70.7,55.2,37.1,34.9


In [8]:
train_df = pd.get_dummies(train_df,columns=['patient_race','payer_type','breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment',],dtype=int)

In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13173 entries, 12275 to 1894
Columns: 238 entries, patient_age to metastatic_first_novel_treatment_PEMBROLIZUMAB
dtypes: float64(137), int64(101)
memory usage: 24.0 MB


In [10]:
train_df.isna().sum()

patient_age                                          0
bmi                                               9071
population                                           0
density                                              0
age_median                                           0
                                                  ... 
metastatic_cancer_diagnosis_code_C7982               0
metastatic_cancer_diagnosis_code_C7989               0
metastatic_cancer_diagnosis_code_C799                0
metastatic_first_novel_treatment_OLAPARIB            0
metastatic_first_novel_treatment_PEMBROLIZUMAB       0
Length: 238, dtype: int64

In [11]:
train_df['bmi'].describe()

count   4102.0
mean      29.2
std        5.8
min       15.0
25%       24.8
50%       28.6
75%       33.0
max       97.0
Name: bmi, dtype: float64

In [12]:
train_df["bmi"].fillna(29.2, inplace=True)


In [ ]:
def fill_nas_mean_value(df: pd.DataFrame):
    if df.isnull().sum().sum() != 0:
        na_df = (df.isnull().sum() / len(df)) * 100
        na_df = na_df.drop(na_df[na_df == 0].index).sort_values(ascending=False)
        missing_data = pd.DataFrame({'Missing Ratio %' :na_df})
        missing_data_more_than_0_percent = missing_data[missing_data['Missing Ratio %'] > 0.0]
        print(missing_data_more_than_0_percent.index)
        for columnName in missing_data_more_than_0_percent.index:
          mean_value = df[columnName].mean()
          print('Column Name ',columnName)
          print('mean value ', mean_value)
          df[columnName].fillna(value=mean_value, inplace=True)


fill_nas_mean_value(train_df)

In [15]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler(quantile_range=(25, 75)).set_output(transform="pandas")

X_train_scaled = scaler.fit_transform(train_df,y_train)
X_train_scaled.head()

,patient_age,bmi,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,...,metastatic_cancer_diagnosis_code_C7970,metastatic_cancer_diagnosis_code_C7971,metastatic_cancer_diagnosis_code_C7972,metastatic_cancer_diagnosis_code_C798,metastatic_cancer_diagnosis_code_C7981,metastatic_cancer_diagnosis_code_C7982,metastatic_cancer_diagnosis_code_C7989,metastatic_cancer_diagnosis_code_C799,metastatic_first_novel_treatment_OLAPARIB,metastatic_first_novel_treatment_PEMBROLIZUMAB
12275,0.6,0.0,0.1,0.0,0.3,0.0,0.0,-0.2,-0.2,0.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2416,-1.2,0.0,1.9,10.8,-0.8,1.2,0.2,0.6,0.4,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11828,-0.1,0.0,-0.1,0.1,-0.1,-0.7,0.2,0.7,-0.2,-1.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8802,-0.3,0.0,-0.6,-0.4,0.8,-1.4,-0.3,-0.3,-0.6,-0.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9237,-1.0,0.0,-0.1,0.2,-0.4,-0.1,1.2,1.1,-0.4,-1.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
from sklearn.decomposition import PCA
pca = PCA(n_components=150)
transformed_data = pca.fit_transform(X_train_scaled)
pcaXdata = pd.DataFrame(transformed_data)
pcaXdata.head()

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-6.5,-2.9,-0.6,0.6,-0.1,-2.1,-0.6,0.2,1.4,0.7,...,0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0
1,0.5,-3.5,-0.9,6.3,2.7,10.7,1.0,-2.3,-2.7,2.2,...,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0
2,-6.8,-0.3,-0.3,-2.0,0.1,2.6,1.7,1.2,1.3,-0.5,...,-0.0,-0.0,-0.0,-0.0,0.1,0.0,0.0,0.0,-0.0,-0.0
3,-6.6,-2.2,-0.6,-0.2,-0.2,-1.8,1.0,-0.9,2.1,0.9,...,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0
4,-3.0,0.9,-1.3,-1.3,0.2,3.0,0.1,2.5,1.3,-1.5,...,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
